# Assignment 2: Training the Fully Recurrent Network

*Author:* Thomas Adler

*Copyright statement:* This  material,  no  matter  whether  in  printed  or  electronic  form,  may  be  used  for  personal  and non-commercial educational use only.  Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.


## Exercise 1: Data generation

There are two classes, both occurring with probability 0.5. There is one input unit. Only the first sequence element conveys relevant information about the class. Sequence elements at positions $t > 1$ stem from a Gaussian with mean zero and variance 0.2. The first sequence element is 1.0 (-1.0) for class 1 (2). Target at sequence end is 1.0 (0.0) for class 1 (2)

Write a function `generate_data` that takes an integer `T` as argument which represents the sequence length. Seed the `numpy` random generator with the number `0xDEADBEEF`. Implement the [Python3 generator](https://docs.python.org/3/glossary.html#term-generator) pattern and produce data in the way described above. The input sequences should have the shape `(T, 1)` and the target values should have the shape `(1,)`.

In [3]:
%matplotlib inline
import numpy as np
from scipy.special import expit as sigmoid
import matplotlib.pyplot as plt

class FullyRecurrentNetwork(object):
    def __init__(self, D, I, K):
        self.W = np.random.uniform(-0.01, 0.01, (I, D))
        self.R = np.random.uniform(-0.01, 0.01, (I, I))
        self.V = np.random.uniform(-0.01, 0.01, (K, I))
    
    def forward(self, x, y):
        # helper function for numerically stable loss
        def f(z):
            return np.log1p(np.exp(-np.absolute(z))) + np.maximum(0, z)
        
        # infer dims
        T, D = x.shape
        K, I = self.V.shape

        # init result arrays
        self.x = x
        self.y = y
        self.a = np.zeros((T, I))

        # iterate forward in time 
        # trick: access model.a[-1] in first iteration
        for t in range(T):
            self.a[t] = np.tanh(self.W @ x[t] + self.R @ self.a[t-1])
            
        self.z = model.V @ self.a[t]
        return y * f(-self.z) + (1-y) * f(self.z)

T, D, I, K = 10, 3, 5, 1
model = FullyRecurrentNetwork(D, I, K)
model.forward(np.random.uniform(-1, 1, (T, D)), 1)

def generate_data(T):
    
    np.random.seed(0xDEADBEEF)

    while True:
        # Randomly assign class (0.5 probability for each)
        class_label = np.random.choice([1, 2])

        sequence = np.zeros((T, 1))

        if class_label == 1:
            sequence[0, 0] = 1.0
            target = np.array([1.0])
        else:
            sequence[0, 0] = -1.0
            target = np.array([0.0])

        # Subsequent elements are Gaussian noise
        sequence[1:, 0] = np.random.normal(loc=0.0, scale=np.sqrt(0.2), size=T-1)

        yield sequence, target
        
data = generate_data(2)

## Exercise 2: Gradients for the network parameters
Compute gradients of the total loss 
$$
L = \sum_{t=1}^T L(t), \quad \text{where} \quad L(t) = L(z(t), y(t))
$$
w.r.t. the weights of the fully recurrent network. To this end, find the derivative of the loss w.r.t. the logits and hidden pre-activations first, i.e., 
$$
\psi^\top(t) = \frac{\partial L}{\partial z(t)} \quad \text{and} \quad \delta^\top(t) = \frac{\partial L}{\partial s(t)}.
$$
With the help of these intermediate results you should be able to compute the gradients w.r.t. the weights, i.e., $\nabla_W L, \nabla_R L, \nabla_V L$. 

*Hint: Take a look at the computational graph from the previous assignment to see the functional dependencies.*

*Remark: Although we only have one label at the end of the sequence, we consider the more general case of evaluating a loss at every time step in this exercise (many-to-many mapping).*


# Solution


1. Gradient with respect to logits $ z(t) $:
$$
\psi^\top(t) = \frac{\partial L}{\partial z(t)} = \frac{\partial L(t)}{\partial z(t)}.
$$

2. Gradient with respect to hidden pre-activations $ s(t) $:
Using the chain rule:
$$
\delta^\top(t) = \frac{\partial L}{\partial s(t)} = \psi^\top(t) \frac{\partial z(t)}{\partial s(t)} + \delta^\top(t+1) \frac{\partial h(t+1)}{\partial s(t)}.
$$
3. 
$$
    \frac{\partial z(t)}{\partial s(t)} &= V^\top \phi'(s(t)), 
    \frac{\partial h(t+1)}{\partial s(t)} &= R^\top \phi'(s(t)).
$$

Therefore:

$$
\delta^\top(t) = \psi^\top(t) V^\top \phi'(s(t)) + \delta^\top(t+1) R^\top \phi'(s(t)).
1$$

# Gradients with Respect to Weights

1. Gradient with respect to $ W $:
$$
\nabla_W L = \sum_{t=1}^T \frac{\partial L}{\partial s(t)} \frac{\partial s(t)}{\partial W}.
$$
Since $ \frac{\partial s(t)}{\partial W} = x(t)^\top $, we have:
$$
\nabla_W L = \sum_{t=1}^T \delta(t) x(t)^\top.
$$

2. Gradient with respect to $ R $:
$$
\nabla_R L = \sum_{t=1}^T \frac{\partial L}{\partial s(t)} \frac{\partial s(t)}{\partial R}.
$$
Since $ \frac{\partial s(t)}{\partial R} = h(t-1)^\top $, we have:
$$
\nabla_R L = \sum_{t=1}^T \delta(t) h(t-1)^\top.
$$

\paragraph{3. Gradient with respect to $ V $:}
$$
\nabla_V L = \sum_{t=1}^T \frac{\partial L}{\partial z(t)} \frac{\partial z(t)}{\partial V}.
$$
Since $ \frac{\partial z(t)}{\partial V} = h(t)^\top $, we have:
$$
\nabla_V L = \sum_{t=1}^T \psi(t) h(t)^\top.
$$

# Backpropagation Through Time (BPTT)

The computation of $ \delta(t) $ involves contributions from $ \delta(t+1) $. This is handled iteratively:
\begin{itemize}
    \item Start at $ t = T $ with:
    $$
    \delta(T) = \psi^\top(T) V^\top \phi'(s(T)).
    $$
    \item Propagate backward to compute $ \delta(t) $ for $ t = T-1, T-2, \dots, 1 $.
\end{itemize}

# Final Results

The gradients are:
$$
\nabla_W L = \sum_{t=1}^T \delta(t) x(t)^\top,
$$
$$
\nabla_R L = \sum_{t=1}^T \delta(t) h(t-1)^\top,
$$
$$
\nabla_V L = \sum_{t=1}^T \psi(t) h(t)^\top,
$$
where $ \psi(t) $ and $ \delta(t) $ are computed recursively.



## Exercise 3: The backward pass
Write a function `backward` that takes a model `self` as argument. The function should compute the gradients of the loss with respect to all model parameters and store them to `self.dW`, `self.dR`, `self.dV`, respectively. 

In [5]:
def backward(self):
    # helper function for gradient of the loss
    def df(z):
        return -1 / (1 + np.exp(np.absolute(z)))

        # get dimensions
    T, D = self.x.shape
    K, I = self.V.shape

    # initialize gradient matrices
    self.dW = np.zeros_like(self.W)  # (I, D)
    self.dR = np.zeros_like(self.R)  # (I, I)
    self.dV = np.zeros_like(self.V)  # (K, I)

    # compute gradient of loss with respect to final output
    dz = self.y * df(-self.z) + (1-self.y) * df(self.z)  # (K,)

    # gradient for V (output layer)
    self.dV = np.outer(dz, self.a[-1])  # (K, I)

    # initialize gradient of hidden state
    da_next = np.zeros(I)  # (I,)

    # backpropagate through time
    for t in reversed(range(T)):
        # current hidden state
        a_t = self.a[t]  # (I,)

        # if it's the last timestep, add gradient from output
        if t == T-1:
            da = self.V.T @ dz  # (I,)
        else:
            da = np.zeros(I)  # (I,)

        # add gradient from next timestep
        da += da_next  # (I,)

        # gradient through tanh
        dtanh = (1 - a_t**2) * da  # (I,)

        # gradients for W and R
        self.dW += np.outer(dtanh, self.x[t])  # (I, D)
        self.dR += np.outer(dtanh, self.a[t-1])  # (I, I)

        # gradient for next timestep
        da_next = self.R.T @ dtanh  # (I,)
    

FullyRecurrentNetwork.backward = backward
model.backward()

## Exercise 4: Gradient checking
Write a function `grad_check` that takes a model `self`, a float `eps` and another float `thresh` as arguments and computes the numerical gradients of the model parameters according to the approximation
$$
f'(x) \approx \frac{f(x + \varepsilon) - f(x - \varepsilon)}{2 \varepsilon}.
$$
If any of the analytical gradients are farther than `thresh` away from the numerical gradients the function should throw an error. 

In [ ]:
def grad_check(self, eps, thresh):
    ########## YOUR SOLUTION HERE ##########

FullyRecurrentNetwork.grad_check = grad_check
model.grad_check(1e-7, 1e-7)

## Exercise 5: Parameter update

Write a function `update` that takes a model `self` and a float argument `eta`, which represents the learning rate. The method should implement the gradient descent update rule $\theta \gets \theta - \eta \nabla_{\theta}L$ for all model parameters $\theta$.

In [ ]:
def update(self, eta):
    ########## YOUR SOLUTION HERE ##########

FullyRecurrentNetwork.update = update
model.update(0.001)

## Exercise 6: Network training

Train the fully recurrent network with 32 hidden units. Start with input sequences of length one and tune the learning rate and the number of update steps. Then increase the sequence length by one and tune the hyperparameters again. What is the maximal sequence length for which the fully recurrent network can achieve a performance that is better than random? Visualize your results. 

In [ ]:
########## YOUR SOLUTION HERE ##########

## Exercise 7: The Vanishing Gradient Problem

Analyze why the network is incapable of learning long-term dependencies. Show that $\|\frac{\partial a(T)}{\partial a(1)}\|_2 \leq \|R\|_2^{T-1}$ , where $\|\cdot\|_2$ is the spectral norm, and discuss how that affects the propagation of error signals through the time dimension of the network. 

*Hint: Use the fact that the spectral norm is submultiplicative for square matrices, i.e. $\|AB\|_2 \leq \|A\|_2\|B\|_2$ if $A$ and $B$ are both square.*

########## YOUR SOLUTION HERE ##########